In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%%capture --no-stderr
!pip install python-dotenv langchain-core langchain-openai langchain-chroma

In [ ]:
# 환경변수 설정

In [ ]:
# 라이브러리 불러오기
from dotenv import load_dotenv
import os
from langchain_openai import OpenAI

In [ ]:
# .env 파일에서 환경 변수 로드
load_dotenv("/content/.env")
# 환경 변수에서 API 키 가져오기
api_key = os.getenv("OPENAI_API_KEY")
# 오픈AI 대규모 언어 모델 초기화
llm = OpenAI()

# **문자열 프롬프트 템플릿**

In [ ]:
# 라이브러리 불러오기
from langchain_core.prompts import PromptTemplate
# 주어진 주제에 대한 조언을 요청하는 프롬프트 템플릿 정의
prompt_template = PromptTemplate.from_template("주제 {topic}에 대해 금융 관련 짧은 조언을 해주세요.")
# 투자 주제로 프롬프트 템플릿 호출
prompt+template.invoke({"topic": "투자"})

# **챗 프롬프트 템플릿**

In [ ]:
# 라이브러리 불러오기
from langchain_core.prompts import ChatPromptTemplate
# 챗 프롬프트 템플릿 정의: 사용자와 시스템 간의 메시지 포함
prompt_template = ChatPromptTemplate.from_messages([
    ("system": "당신은 유능한 금융 조언가입니다."),
    ("user", "주제 {topic}에 대해 금융 관련 조언을 해주세요.")
])

# 주식 주제로 챗 프롬프트 템플릿 호출
prompt_template.invoke({"topic": "주식"})

# **메시지 자리 표시자 템플릿**

In [ ]:
# 라이브러리 불러오기
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage

# 방법 1 메시지 자리 표시자를 포함한 챗 프롬프트 템플릿 정의
prompt_template = ChatPromptTemplate.from_messages([
    ("system", "당신은 유능한 금융 조언가입니다."),
    MessagePlaceholder("msgs")
])

# 메시지 리스트를 msgs 자리 표시자에 전달하여 호출

# HumanMessage는 LangChain에서 제공하는 메시지 타입 중 하나
# 대화형 모델에서 사람이 입력한 메시지를 나타내는 객체
# LLM이나 체인에 넘길 때, 메시지를 역할(role) 기반으로 관리하기 위해 사용
# 이 객체는 단순 문자열이 아니라 역할(role) + 내용(content) + 추가 메타데이터를 담을 수 있어
# {"role": "user", "content": "안녕하세요!"}
prompt_template.invoke({"msgs": [HumanMessage(content="안녕하세요!")]})


In [ ]:
# 방법2 MessagesPlaceholder 클래스를 사용하지 않고 비슷한 작업 수행
prompt_template = ChatPromptTemplate.from_messages([
    ("system", "당신은 유능한 금융 조언가입니다."),
    # 여기서 msgs가 자리 표시자로 사용된다
    ("placeholder", "{mesg}")
])

# 메시지 리스트를 msgs 자리 표시자에 전달하여 호출
prompt_template.invoke({"msgs": [HumanMessage(content="안녕하세요!")]})

# **PromptTemplate를 이용한 퓨샷 프롬프트**

In [ ]:
# 라이브러리 불러오기
from langchain_core.prompts import PromptTemplate
# 질문과 답변을 포맷하는 프롬프트 템플릿 정의
example_prompt = PromptTemplate.from_template("질문: {question}\n답변: {answer}")

In [ ]:
# 퓨샷 예제 목록 생성
examples = [
    {
        "question": "주식 투자와 예금 중 어느 것이 더 수익률이 높은가?"
        "answer": """
        후속 질문이 필요한가요: 네.
        후속 질문: 주식 투자의 평균 수익률은 얼마인가요?
        중간 답변: 주식 투자의 평균 수익률은 연 7%입니다.
        후속 질문: 예금의 평균 이자율은 얼마인가요?
        중간 답변: 예금의 평균 이자율은 연 1%입니다.
        따라서 최종 답변은: 주식 투자
        """,
    },
    {
        "question": "부동산과 채권 중 어느 것이 더 안정적인 투자처인가?",
        "answer": """
        후속 질문이 필요한가요: 네.
        후속 질문: 부동산 투자의 위험도는 어느 정도인가요?
        중간 답변: 부동산 투자의 위험도는 중간 수준입니다.
        후속 질문: 채권의 위험도는 어느 정도인가요?
        중간 답변: 채권의 위험도는 낮은 편입니다.
        따라서 최종 답변은: 채권
        """,
    },
]

In [ ]:
print(example_prompt.invoke(examples[0].to_string()))

# **FewShotPromptTemplate 를 이용한 퓨샷 프롬프트**

In [ ]:
# 라이브러리 불러오기
from langchain_core.prompts import FewShotPromptTemplate
# FewShotPromptTmeplate 생성
prompt = FewShotPromptTemplate(
    # 위에서 정의한 Q&A 샘플
    # 예제 프롬프트(example_prompt + examples)
    # 미리 정의해둔 Q&A 예시들이 프롬프트 앞부분에 붙는다
    # 일종의 샘플 답안 / 학습 데이터 역할을 한다
    examples=examples,
    # 각 예제를 어떻게 출력할지
    example_prompt=example_prompt,
    # 마지막에 진짜 입력 붙이는 자리
    # 실제 질문(suffix + input)
    # 마지막에 우리가 진짜 LLM에 물어볼 질문이 붙는다
    suffix="질문: {input}",
    # suffix 안에 들어갈 변수
    input_variables=["input"],
)

# 부동산 투자 주제로 프롬프트 호출 및 출력
print(
    prompt.invoke({"input": "부동산 투자의 장점은 무엇인가?"}).to_string()
)

# LLM은 전체 프롬프트(예제 + 실제 질문)를 입력으로 받고
# 예제들을 패턴 참고 자료로 삼아서 새로운 질문에 대해 비슷한 형식으로 답변하려 한다